In [1]:
import tensorflow as tf,cv2,os,time
import numpy as np
from tensorflow.python.keras.utils.data_utils import get_file
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

[]


In [9]:
np.random.seed(0)

class Detector:
    def __init__(self):
        pass

    def readclasses(self,classfilepath):
        with open(classfilepath,'r') as f:
            self.classeslist = f.read().splitlines()

        self.colorlist = np.random.uniform(low=0, high=255, size=(len(self.classeslist),3))

        print(len(self.classeslist),len(self.colorlist))

    def downloadmodel(self,modelURL):

        filename = os.path.basename(modelURL)
        self.modelname = filename[:filename.index('.')]

        print(filename)
        print(self.modelname)

        self.cachedir = "./pretrained_model"
        os.makedirs(self.cachedir,exist_ok=True)
        get_file(fname=filename, origin=modelURL, cache_dir=self.cachedir, cache_subdir="checkpoints", extract=True) 

    def loadmodel(self):
        print("Loading Model:"+self.modelname)
        tf.keras.backend.clear_session()
        self.model =  tf.saved_model.load(os.path.join(self.cachedir, "checkpoints", self.modelname, "saved_model"))
        print("Model:"+self.modelname+"Loaded successfully")

    def createboundingbox(self,image, threshold=0.5):
        inputTensor = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
        inputTensor = tf.convert_to_tensor(inputTensor, dtype=tf.uint8)
        inputTensor = inputTensor[tf.newaxis,...]
        detections = self.model(inputTensor)
        bboxs = detections['detection_boxes'][0].numpy()
        classindexes = detections['detection_classes'][0].numpy().astype(np.int32)
        classscores = detections['detection_scores'][0].numpy()

        imh, imw, imc = image.shape

        bboxidx = tf.image.non_max_suppression(bboxs, classscores, max_output_size=50, iou_threshold=0.5, score_threshold=0.5)

        print(bboxidx)

        if len(bboxidx) != 0:
            for i in bboxidx:
                bbox = tuple(bboxs[i].tolist())
                classconfidence = round(100*classscores[i])     
                classindex = classindexes[i]
                classlabeltext = self.classeslist[classindex]
                classcolor = self.colorlist[classindex]
                displaytext = '{}: {}%'.format(classlabeltext, classconfidence)

                ymin, xmin, ymax, xmax = bbox
                xmin, xmax, ymin, ymax = (xmin * imw, xmax * imw, ymin * imh, ymax * imh)
                xmin, xmax, ymin, ymax = int(xmin), int(xmax), int(ymin), int(ymax)
                cv2.rectangle(image, (xmin, ymin), (xmax, ymax), classcolor, thickness=1) 
                cv2.putText(image, displaytext, (xmin,ymin - 10), cv2.FONT_HERSHEY_PLAIN, 1, classcolor, 2)
                
        return image

    def predictimage(self, imagepath, threshold = 0.5):

        image = cv2.imread(imagepath)
        bboximage = self.createboundingbox(image, threshold)
        cv2.imwrite(self.modelname + ".jpg", bboximage)
        cv2.imshow("Result", image)
        cv2.waitKey(0) 
        cv2.destroyAllWindows()
